In [92]:
from datetime import datetime
import numpy as np
import os
import pandas as pd
import tensorflow as tf
from sklearn.metrics import roc_auc_score
import random
from tensorflow.contrib.layers import fully_connected 
import math
os.environ["CUDA_VISIBLE_DEVICES"]='-1'
timewindow=72

#sepsis auclos 59.20153714773698  non_sepsis 37.37243960303405

# 1、导入数据并且清理数据

In [93]:
def load_data(filename):
    setAB_data=np.load(filename)
    setAB_X_t=setAB_data['X_t']   
    setAB_X_last_obsv=setAB_data['X_last_obsv'] 

    setAB_X_t_mask=setAB_data['X_t_mask']
    setAB_deltaT_t=setAB_data['deltaT_t']
    setAB_label=setAB_data['label']

    return setAB_X_t,setAB_X_last_obsv,setAB_X_t_mask,setAB_deltaT_t, setAB_label


def pad_matrix(x):
    n = len(x)
    t = timewindow
    d = x[0].shape[1]
    
    ret = np.zeros([n, t, d], dtype=float)   
    length=[len(i) for i in x]
    
    for i in range(len(x)):
        if len(x[i])<t:
            ret[i, :length[i],:] = x[i]
        else:
            ret[i,:,:] = x[i][length[i]-t:length[i],:]         
    return ret


def get_Normalization(X):

    X_mean=np.array([84.54736749058092, 97.19735567887244, 36.97384942767003, 123.79455870514964, 82.39844634983528, 
                     63.82168093649573, 18.720210390214252, 33.01391751459007, -0.6869669057468529, 24.07165352824123,
                     0.5629699098557109, 7.378730326999917, 41.08205268057132, 92.6638691105628, 269.71012201748755, 
                     23.989203500427596, 103.83241100807658, 7.578440945745791, 105.83631589184984, 1.5132853634707293, 
                     1.7982557651991606, 137.10754340757103, 2.666527938225818, 2.0523013086498567, 3.5532881933995384, 
                     4.138466653390937, 2.1437651299155416, 8.152533277169335, 30.79291672353087, 10.43255372477224, 
                     41.26944061491329, 11.458261650095405, 284.57097280966764, 196.1678411769487, 62.02033203772457, 
                     0.5577890176790509, 0.49908857795492095, 0.5009114220450791, -55.728074496237774, 26.93293913918774]) 
    
    X_std=np.array([17.357143533110786, 2.9197410787621823, 0.769008502635353, 23.226370766088404, 16.35175339480381, 
                   13.990094006590905, 5.096634488705786, 7.901376008166723, 4.288142385817428, 4.3942700623947095, 
                   12.460897950458373, 0.07483160020802701, 9.327999041120131, 10.841774762519147, 881.945227133347,
                   20.067454723313723, 125.78376567914378, 2.420980582249712, 5.856066574937517, 1.8016280879794837,
                   3.555075159474239, 51.31864530748081, 2.5840546941276425, 0.4008345314433298, 1.438200307219221, 
                   0.6449469067770479, 4.358151926253383, 24.455704359390893, 5.491328159692898, 1.9666320193417495, 
                   26.26311427074314, 7.737594668485472, 150.0117749185703, 103.8931301983612, 16.41032814994427, 
                   0.49664920158567677, 0.4999991693091656, 0.4999991693091656, 161.2959477283614, 28.699759200381497])
    
    num_patient=X.shape[0]    
    for i in range(num_patient):    
        X[i] = (X[i] - X_mean)/X_std
    return X

def get_minibatch(index,mb_size,X_t,X_last_obsv,X_t_mask,deltaT_t,label):   
    
    X_t = X_t[index:index+mb_size]
    X_last_obsv=X_last_obsv[index:index+mb_size]
    X_t_mask = X_t_mask[index:index+mb_size]
    deltaT_t = deltaT_t[index:index+mb_size]
    label = label[index:index+mb_size]
    
    seq_len=np.array([len(X_t[i]) for i in range(len(X_t))])
    
    seq_len=np.where(seq_len>timewindow,timewindow,seq_len)
    
    X_t=pad_matrix(get_Normalization(X_t))
    X_last_obsv=pad_matrix(get_Normalization(X_last_obsv))

    X_t_mask=pad_matrix(X_t_mask)
    deltaT_t=pad_matrix(deltaT_t)
    label=pad_matrix(label)

    label_mask=np.zeros_like(label, dtype=int)
    
    for i in range(label.shape[0]):
        label_mask[i,:seq_len[i],:]=1
        
    return  (X_t, X_last_obsv,X_t_mask , deltaT_t,label,seq_len,label_mask)

def get_val_data(filename):
    
    setAB_data=np.load(filename)
    setAB_X_t=setAB_data['X_t']   
    setAB_X_last_obsv=setAB_data['X_last_obsv'] 

    setAB_X_t_mask=setAB_data['X_t_mask']
    setAB_deltaT_t=setAB_data['deltaT_t']
    setAB_label=setAB_data['label']
    
    seq_len=np.array([len(setAB_X_t[i]) for i in range(len(setAB_X_t))])
    
    seq_len=np.where(seq_len>timewindow,timewindow,seq_len)
    
    X_t=pad_matrix(get_Normalization(setAB_X_t))
    X_last_obsv=pad_matrix(get_Normalization(setAB_X_last_obsv))

    X_t_mask=pad_matrix(setAB_X_t_mask)
    deltaT_t=pad_matrix(setAB_deltaT_t)
    label=pad_matrix(setAB_label)
    
    label_mask=np.zeros_like(label, dtype=int)
    
    for i in range(label.shape[0]):
        label_mask[i,:seq_len[i],:]=1
    
    return (X_t,X_last_obsv, X_t_mask, deltaT_t,label,seq_len,label_mask)
    
def suffle_data(X_t,X_last_obsv,X_t_mask,deltaT_t,label):
    batch=X_t.shape[0]
    random_index=np.arange(batch).tolist()
    shuffle=random.sample(random_index,batch) 
    X_t=X_t[shuffle]
    X_last_obsv=X_last_obsv[shuffle]
    X_t_mask=X_t_mask[shuffle]
    deltaT_t=deltaT_t[shuffle]
    label=label[shuffle]
    return X_t,X_last_obsv,X_t_mask,deltaT_t,label   

# 2、设计网络模型

In [106]:
def FilterLinear(input1,in_features, out_features, filter_square_matrix):
    stdv = 1./ math.sqrt(in_features)
    weight=tf.Variable(tf.random.uniform((out_features,in_features),minval=-stdv, maxval=stdv,dtype=tf.float32))  
    bias=tf.Variable(tf.random.uniform((out_features,),minval=-stdv, maxval=stdv,dtype=tf.float32))
    
    temp=filter_square_matrix*weight
    result=tf.add(tf.matmul(input1,temp),bias)
    
    return result

def FCNet(inputs,h_dim,o_fn):     
    layers  = fully_connected (inputs=inputs, num_outputs=h_dim, activation_fn=o_fn)
    return layers


class Model_GRUD:
    def __init__(self, sess,name,input_size):
        self.sess=sess
        self.name=name  
        
        self.input_size=input_size
        self.delta_size = input_size      
        self.mask_size = input_size                   
        self.hidden_size=input_size
               
        self.forward()
        

    def grud_cell(self, x, x_last_obsv, x_mean, h, mask, delta):
        
        input_temp= tf.stack((self.zeros_matrix,FilterLinear(delta,self.delta_size, self.delta_size, self.identity)),axis=1)

     
        delta_x = tf.exp(-tf.reduce_max(input_temp,reduction_indices=1))
        
        hidden_temp=tf.stack((self.zeros_matrix,FCNet(delta,self.delta_size,None)),axis=1)    
        delta_h = tf.exp(-tf.reduce_max(hidden_temp,reduction_indices=1))    
        
        
        x = mask * x + (1 - mask) * (delta_x * x_last_obsv + (1 - delta_x) * x_mean)       
        h = delta_h * h

        combined = tf.concat((x, h, mask), axis=1)

        
        weight=tf.Variable(tf.random.normal([self.input_size+self.hidden_size+self.mask_size,self.hidden_size], 
                                                      stddev=0.01))  
        bias=tf.Variable(tf.zeros([self.hidden_size]))
        z=tf.nn.sigmoid(tf.add(tf.matmul(combined,weight),bias))
        
        weight2=tf.Variable(tf.random.normal([self.input_size+self.hidden_size+self.mask_size,self.hidden_size], 
                                                       stddev=0.01)) 
        bias2=tf.Variable(tf.zeros([self.hidden_size]))  
        
        r=tf.nn.sigmoid(tf.add(tf.matmul(combined,weight2),bias2))
               
        combined_r = tf.concat((x, r * h, mask), axis=1)

        weight3=tf.Variable(tf.random.normal([self.input_size+self.hidden_size+self.mask_size,self.hidden_size], 
                                                       stddev=0.01))   
        bias3=tf.Variable(tf.zeros([self.hidden_size]))
        
        h_tilde=tf.nn.tanh(tf.add(tf.matmul(combined_r,weight3),bias3))
                 
        h = (1 - z) * h + z * h_tilde

        return h
    
    def forward(self):      

        self.X_t         = tf.placeholder(tf.float32, shape=[None,None, 40], name='X_t')
        self.X_last_obsv = tf.placeholder(tf.float32, shape=[None,None, 40], name='X_last_obsv')
        self.X_t_mask    = tf.placeholder(tf.float32, shape=[None,None, 40], name='mask')     
        self.deltaT_t    = tf.placeholder(tf.float32, shape=[None,None, 40], name='timestamp') 
        
        self.labels      = tf.placeholder(tf.float32, shape=[None,None, 1], name='label')
        self.label_mask  = tf.placeholder(tf.float32, shape=[None,None, 1], name='label') 
        
        
        self.batch_size  = tf.placeholder(tf.int32, [], name='batch_size')
        self.seq_length  = tf.placeholder(tf.float32, shape=[None,], name='label')
        
        self.max_length  = timewindow
        
        self.identity    = tf.eye((self.input_size)) 
        self.zeros_matrix= tf.zeros((self.batch_size,self.delta_size))         
        self.X_mean      = tf.zeros((self.batch_size,self.input_size))         
        Hidden_State     = tf.zeros((self.batch_size, self.hidden_size))

        outputs_final = [] 
        
        for i in range(self.max_length):
            Hidden_State =self.grud_cell(tf.squeeze(self.X_t[:, i:i + 1, :], axis=1),
                                     tf.squeeze(self.X_last_obsv[:, i:i + 1, :], axis=1),
                                     tf.squeeze(self.X_mean),
                                     Hidden_State,
                                     tf.squeeze(self.X_t_mask[:, i:i + 1, :], axis=1),
                                     tf.squeeze(self.deltaT_t[:, i:i + 1, :], axis=1))
            
        
            outputs_temp = FCNet(Hidden_State,1,tf.nn.sigmoid)
            outputs_final.append(outputs_temp) 
            
        self.outputs= tf.stack(outputs_final, axis=1)            
           
        ##加上label的mask，对于那些人为补齐的序列不去计算它的损失
        self.outputs2=self.label_mask*self.outputs
        

        ## 加上1e-20是为了防止log(0)bug,导致loss为nan
        self.loss = -tf.reduce_sum(tf.reduce_sum(self.labels*tf.math.log(tf.add(self.outputs2,1e-20))+        
                                (1-self.labels)*tf.math.log(1-self.outputs2),1))/ (tf.reduce_sum(self.seq_length))
             
        self.solver = tf.train.AdamOptimizer(learning_rate=0.001).minimize(self.loss)                                             
 

    def train(self, batch_data):                 
        _,current_loss=self.sess.run([self.solver, self.loss], 
                                    feed_dict={self.X_t: batch_data[0],         self.X_last_obsv: batch_data[1], 
                                               self.X_t_mask:batch_data[2],     self.deltaT_t: batch_data[3],
                                               self.labels:batch_data[4],       self.batch_size:batch_data[0].shape[0],
                                               self.seq_length:batch_data[5],   self.label_mask:batch_data[6]})
        return current_loss

    def validation(self, val_data): 
        current_loss=self.sess.run(self.loss, 
                                    feed_dict={self.X_t: val_data[0],            self.X_last_obsv: val_data[1], 
                                               self.X_t_mask:val_data[2],        self.deltaT_t: val_data[3],
                                               self.labels:val_data[4],          self.batch_size:val_data[0].shape[0],
                                               self.seq_length:val_data[5],      self.label_mask:val_data[6]})
        return current_loss
        
    def predict(self, test_data):  
        output=self.sess.run(self.outputs, 
                             feed_dict={self.X_t: test_data[0],                  self.X_last_obsv: test_data[1], 
                                        self.X_t_mask:test_data[2],              self.deltaT_t: test_data[3],
                                        self.batch_size:test_data[0].shape[0]})
        return output   

# 3、下载数据

In [96]:
X_t,X_last_obsv,X_t_mask,deltaT_t,label = load_data('/data/jiawenxiao/physionet0727/train_data.npz')
val_data=get_val_data('/data/jiawenxiao/physionet0727/val_data.npz')

# 4、 训练模型

In [110]:
input_size=40
tf.reset_default_graph()
config = tf.ConfigProto()
sess = tf.Session(config=config)
model= Model_GRUD(sess,'GRUD',input_size)
sess.run(tf.global_variables_initializer())

batch_size=256
iters=7
batch=X_t.shape[0]
  
for itr in range(1,iters):
    X_t_shuffle,X_last_obsv_shuffle,X_t_mask_shuffle,deltaT_t_shuffle,label_shuffle=suffle_data(X_t,
                                                                 X_last_obsv,X_t_mask,deltaT_t,label)
    total_loss=0  
    num=0
    for i in range(0,batch-batch_size,batch_size):
        batch_data=get_minibatch(i,batch_size,X_t_shuffle,X_last_obsv_shuffle,X_t_mask_shuffle,deltaT_t_shuffle,label_shuffle)
        curr_loss= model.train(batch_data)  
        total_loss+=curr_loss
        num+=1     
    val_loss= model.validation(val_data)  
    avg_loss=total_loss/num
    print('|| ITR: ' + str('%04d' % (itr)) + ' | Train Loss: ' +str('%.4f' %(avg_loss))+'  Val Loss '+str('%.4f' %(val_loss)))


|| ITR: 0001 | Train Loss: 0.1002  Val Loss 0.0690
|| ITR: 0002 | Train Loss: 0.0667  Val Loss 0.0667
|| ITR: 0003 | Train Loss: 0.0639  Val Loss 0.0658
|| ITR: 0004 | Train Loss: 0.0618  Val Loss 0.0658
|| ITR: 0005 | Train Loss: 0.0602  Val Loss 0.0649
|| ITR: 0006 | Train Loss: 0.0586  Val Loss 0.0652


# 5、测试数据

In [115]:
def get_filename(path,filetype):
    name =[]
    final_name = []
    for root,dirs,files in os.walk(path):
        for i in files:
            if filetype in i:
                name.append(i.replace(filetype,''))#生成不带‘.csv’后缀的文件名组成的列表
    final_name = [item +'.psv' for item in name]#生成‘.csv’后缀的文件名组成的列表
    return final_name

def save_challenge_predictions(file, scores, labels):
    with open(file, 'w') as f:
        f.write('PredictedProbability|PredictedLabel\n')
        for (s, l) in zip(scores, labels):
            f.write('%g|%d\n' % (s, l))

def pad_matrix_test(x):
    n = 1
    t = timewindow
    d = 40
    ret = np.zeros([n, t, 40], dtype=float)   
    length=len(x)
    if length<t:
        ret[:, :length,:] = x
    else:
        ret[0,:,:] = x[length-t:length,:]         
    return ret

def get_Normalization_test(X):
    X_mean=np.array([84.54736749058092, 97.19735567887244, 36.97384942767003, 123.79455870514964, 82.39844634983528, 
                     63.82168093649573, 18.720210390214252, 33.01391751459007, -0.6869669057468529, 24.07165352824123,
                     0.5629699098557109, 7.378730326999917, 41.08205268057132, 92.6638691105628, 269.71012201748755, 
                     23.989203500427596, 103.83241100807658, 7.578440945745791, 105.83631589184984, 1.5132853634707293, 
                     1.7982557651991606, 137.10754340757103, 2.666527938225818, 2.0523013086498567, 3.5532881933995384, 
                     4.138466653390937, 2.1437651299155416, 8.152533277169335, 30.79291672353087, 10.43255372477224, 
                     41.26944061491329, 11.458261650095405, 284.57097280966764, 196.1678411769487, 62.02033203772457, 
                     0.5577890176790509, 0.49908857795492095, 0.5009114220450791, -55.728074496237774, 26.93293913918774]) 
    
    X_std=np.array([17.357143533110786, 2.9197410787621823, 0.769008502635353, 23.226370766088404, 16.35175339480381, 
                   13.990094006590905, 5.096634488705786, 7.901376008166723, 4.288142385817428, 4.3942700623947095, 
                   12.460897950458373, 0.07483160020802701, 9.327999041120131, 10.841774762519147, 881.945227133347,
                   20.067454723313723, 125.78376567914378, 2.420980582249712, 5.856066574937517, 1.8016280879794837,
                   3.555075159474239, 51.31864530748081, 2.5840546941276425, 0.4008345314433298, 1.438200307219221, 
                   0.6449469067770479, 4.358151926253383, 24.455704359390893, 5.491328159692898, 1.9666320193417495, 
                   26.26311427074314, 7.737594668485472, 150.0117749185703, 103.8931301983612, 16.41032814994427, 
                   0.49664920158567677, 0.4999991693091656, 0.4999991693091656, 161.2959477283614, 28.699759200381497])
    
    X = (X - X_mean)/X_std
    return X


def get_sepsis_score(data, model,threshold): 
    if len(data.shape)==1:
        data=data.reshape(1,-1)

    pd_data=pd.DataFrame(data)
    X_last_obsv= np.array(pd_data.fillna(method='ffill').fillna(0))
    X_t=data.astype(float)
    X_t_mask = 1-np.isnan(X_t).astype('int8')
      
    X_t[np.isnan(X_t)] = 0
    deltaT_t= np.zeros_like(X_t, dtype=int)
    deltaT_t[0,:] = 0
    for i_t in range(1, len(X_t)):
        deltaT_t[i_t,:] = 1 + (1-X_t_mask[i_t,:]) * deltaT_t[i_t-1,:]
    
    X_t_test=pad_matrix_test(get_Normalization_test(X_t))
    X_last_obsv_test=pad_matrix_test(get_Normalization_test(X_last_obsv))

    X_t_mask_test=pad_matrix_test(X_t_mask)
    deltaT_t_test=pad_matrix_test(deltaT_t)
    
    test_data=( X_t_test, X_last_obsv_test,X_t_mask_test,deltaT_t_test)
    
    scores= model.predict(test_data) 
    
    length=len(data)
    seq_len=np.where(length>timewindow,timewindow, length)
    
    scores=scores[:,seq_len-1,:]
    
    labels=(scores>threshold)     
    return (scores, labels)


# 6、测试utility

In [112]:
def load_column(filename, header, delimiter):
    column = []
    with open(filename, 'r') as f:
        for i, l in enumerate(f):
            arrs = l.strip().split(delimiter)
            if i == 0:
                try:
                    j = arrs.index(header)
                except:
                    raise Exception('{} must contain column with header {} containing numerical entries.'.format(filename, header))
            else:
                if len(arrs[j]):
                    column.append(float(arrs[j]))
    return np.array(column)

def compute_auc(labels, predictions, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not 0 <= prediction <= 1:
                warnings.warn('Predictions do not satisfy 0 <= prediction <= 1.')

    # Find prediction thresholds.
    thresholds = np.unique(predictions)[::-1]
    if thresholds[0] != 1:
        thresholds = np.insert(thresholds, 0, 1)
    if thresholds[-1] == 0:
        thresholds = thresholds[:-1]

    n = len(labels)
    m = len(thresholds)

    # Populate contingency table across prediction thresholds.
    tp = np.zeros(m)
    fp = np.zeros(m)
    fn = np.zeros(m)
    tn = np.zeros(m)

    # Find indices that sort the predicted probabilities from largest to
    # smallest.
    idx = np.argsort(predictions)[::-1]

    i = 0
    for j in range(m):
        # Initialize contingency table for j-th prediction threshold.
        if j == 0:
            tp[j] = 0
            fp[j] = 0
            fn[j] = np.sum(labels)
            tn[j] = n - fn[j]
        else:
            tp[j] = tp[j - 1]
            fp[j] = fp[j - 1]
            fn[j] = fn[j - 1]
            tn[j] = tn[j - 1]

        # Update contingency table for i-th largest predicted probability.
        while i < n and predictions[idx[i]] >= thresholds[j]:
            if labels[idx[i]]:
                tp[j] += 1
                fn[j] -= 1
            else:
                fp[j] += 1
                tn[j] -= 1
            i += 1

    # Summarize contingency table.
    tpr = np.zeros(m)
    tnr = np.zeros(m)
    ppv = np.zeros(m)
    npv = np.zeros(m)

    for j in range(m):
        if tp[j] + fn[j]:
            tpr[j] = tp[j] / (tp[j] + fn[j])
        else:
            tpr[j] = 1
        if fp[j] + tn[j]:
            tnr[j] = tn[j] / (fp[j] + tn[j])
        else:
            tnr[j] = 1
        if tp[j] + fp[j]:
            ppv[j] = tp[j] / (tp[j] + fp[j])
        else:
            ppv[j] = 1
        if fn[j] + tn[j]:
            npv[j] = tn[j] / (fn[j] + tn[j])
        else:
            npv[j] = 1

    auroc = 0
    auprc = 0
    for j in range(m-1):
        auroc += 0.5 * (tpr[j + 1] - tpr[j]) * (tnr[j + 1] + tnr[j])
        auprc += (tpr[j + 1] - tpr[j]) * ppv[j + 1]

    return auroc, auprc


def compute_accuracy_f_measure(labels, predictions, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

    # Populate contingency table.
    n = len(labels)
    tp = 0
    fp = 0
    fn = 0
    tn = 0

    for i in range(n):
        if labels[i] and predictions[i]:
            tp += 1
        elif not labels[i] and predictions[i]:
            fp += 1
        elif labels[i] and not predictions[i]:
            fn += 1
        elif not labels[i] and not predictions[i]:
            tn += 1

    # Summarize contingency table.
    if tp + fp + fn + tn:
        accuracy = float(tp + tn) / float(tp + fp + fn + tn)
    else:
        accuracy = 1.0

    if 2 * tp + fp + fn:
        f_measure = float(2 * tp) / float(2 * tp + fp + fn)
    else:
        f_measure = 1.0

    return accuracy, f_measure

def compute_prediction_utility(labels, predictions, dt_early=-12, dt_optimal=-6, dt_late=3.0, max_u_tp=1, min_u_fn=-2, u_fp=-0.05, u_tn=0, check_errors=True):
    # Check inputs for errors.
    if check_errors:
        if len(predictions) != len(labels):
            raise Exception('Numbers of predictions and labels must be the same.')

        for label in labels:
            if not label in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

        for prediction in predictions:
            if not prediction in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

        if dt_early >= dt_optimal:
            raise Exception('The earliest beneficial time for predictions must be before the optimal time.')

        if dt_optimal >= dt_late:
            raise Exception('The optimal time for predictions must be before the latest beneficial time.')

    # Does the patient eventually have sepsis?
    if np.any(labels):
        is_septic = True
        t_sepsis = np.argmax(labels) - dt_optimal     ##########################之前的评分没有dt_optimal这一项
    else:
        is_septic = False
        t_sepsis = float('inf')

    n = len(labels)

    # Define slopes and intercept points for utility functions of the form
    # u = m * t + b.
    m_1 = float(max_u_tp) / float(dt_optimal - dt_early)
    b_1 = -m_1 * dt_early
    m_2 = float(-max_u_tp) / float(dt_late - dt_optimal)
    b_2 = -m_2 * dt_late
    m_3 = float(min_u_fn) / float(dt_late - dt_optimal)
    b_3 = -m_3 * dt_optimal

    # Compare predicted and true conditions.
    u = np.zeros(n)
    for t in range(n):
        if t <= t_sepsis + dt_late:
            # TP
            if is_septic and predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = max(m_1 * (t - t_sepsis) + b_1, u_fp)
                elif t <= t_sepsis + dt_late:
                    u[t] = m_2 * (t - t_sepsis) + b_2
            # FP
            elif not is_septic and predictions[t]:
                u[t] = u_fp
            # FN
            elif is_septic and not predictions[t]:
                if t <= t_sepsis + dt_optimal:
                    u[t] = 0
                elif t <= t_sepsis + dt_late:
                    u[t] = m_3 * (t - t_sepsis) + b_3
            # TN
            elif not is_septic and not predictions[t]:
                u[t] = u_tn

    # Find total utility for patient.
    return np.sum(u)

def evaluate_scores(label_directory, prediction_directory):
    # Set parameters.
    label_header       = 'SepsisLabel'
    prediction_header  = 'PredictedLabel'
    probability_header = 'PredictedProbability'

    dt_early   = -12
    dt_optimal = -6
    dt_late    = 3

    max_u_tp = 1
    min_u_fn = -2
    u_fp     = -0.05
    u_tn     = 0

    # Find label and prediction files.
    label_files = []
    for f in os.listdir(label_directory):
        g = os.path.join(label_directory, f)
        if os.path.isfile(g) and not f.lower().startswith('.') and f.lower().endswith('psv'):
            label_files.append(g)
    label_files = sorted(label_files)

    prediction_files = []
    for f in os.listdir(prediction_directory):
        g = os.path.join(prediction_directory, f)
        if os.path.isfile(g) and not f.lower().startswith('.') and f.lower().endswith('psv'):
            prediction_files.append(g)
    prediction_files = sorted(prediction_files)

    if len(label_files) != len(prediction_files):
        raise Exception('Numbers of label and prediction files must be the same.')

    # Load labels and predictions.
    num_files            = len(label_files)
    cohort_labels        = []
    cohort_predictions   = []
    cohort_probabilities = []

    for k in range(num_files):
        labels        = load_column(label_files[k], label_header, '|')
        predictions   = load_column(prediction_files[k], prediction_header, '|')
        probabilities = load_column(prediction_files[k], probability_header, '|')

        # Check labels and predictions for errors.
        if not (len(labels) == len(predictions) and len(predictions) == len(probabilities)):
            raise Exception('Numbers of labels and predictions for a file must be the same.')

        num_rows = len(labels)

        for i in range(num_rows):
            if labels[i] not in (0, 1):
                raise Exception('Labels must satisfy label == 0 or label == 1.')

            if predictions[i] not in (0, 1):
                raise Exception('Predictions must satisfy prediction == 0 or prediction == 1.')

            if not 0 <= probabilities[i] <= 1:
                warnings.warn('Probabilities do not satisfy 0 <= probability <= 1.')

        if 0 < np.sum(predictions) < num_rows:
            min_probability_positive = np.min(probabilities[predictions == 1])
            max_probability_negative = np.max(probabilities[predictions == 0])

            if min_probability_positive <= max_probability_negative:
                warnings.warn('Predictions are inconsistent with probabilities, i.e., a positive prediction has a lower (or equal) probability than a negative prediction.')

        # Record labels and predictions.
        cohort_labels.append(labels)
        cohort_predictions.append(predictions)
        cohort_probabilities.append(probabilities)

    # Compute AUC, accuracy, and F-measure.
    labels        = np.concatenate(cohort_labels)
    predictions   = np.concatenate(cohort_predictions)
    probabilities = np.concatenate(cohort_probabilities)

    auroc, auprc        = compute_auc(labels, probabilities)
    accuracy, f_measure = compute_accuracy_f_measure(labels, predictions)

    # Compute utility.
    observed_utilities = np.zeros(num_files)
    best_utilities     = np.zeros(num_files)
    worst_utilities    = np.zeros(num_files)
    inaction_utilities = np.zeros(num_files)

    for k in range(num_files):
        labels = cohort_labels[k]
        num_rows          = len(labels)
        observed_predictions = cohort_predictions[k]
        best_predictions     = np.zeros(num_rows)
        worst_predictions    = np.zeros(num_rows)
        inaction_predictions = np.zeros(num_rows)

        if np.any(labels):
            t_sepsis = np.argmax(labels) - dt_optimal                            ###############之前没有dt_optimal这一项
            best_predictions[max(0, t_sepsis + dt_early) : min(t_sepsis + dt_late + 1, num_rows)] = 1
        worst_predictions = 1 - best_predictions

        observed_utilities[k] = compute_prediction_utility(labels, observed_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        best_utilities[k]     = compute_prediction_utility(labels, best_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        worst_utilities[k]    = compute_prediction_utility(labels, worst_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)
        inaction_utilities[k] = compute_prediction_utility(labels, inaction_predictions, dt_early, dt_optimal, dt_late, max_u_tp, min_u_fn, u_fp, u_tn)

    unnormalized_observed_utility = np.sum(observed_utilities)
    unnormalized_best_utility     = np.sum(best_utilities)
    unnormalized_worst_utility    = np.sum(worst_utilities)
    unnormalized_inaction_utility = np.sum(inaction_utilities)
    normalized_observed_utility = (unnormalized_observed_utility - unnormalized_inaction_utility) / (unnormalized_best_utility - unnormalized_inaction_utility)
    return auroc, auprc, accuracy, f_measure, normalized_observed_utility

In [116]:
filetype ='.psv'
test_path='/data/jiawenxiao/physionet0727/test_data/'
output_path='/data/jiawenxiao/physionet0727/test_output/'
test_name=get_filename(test_path,filetype)

cutoff=0.020

for k in cutoff:
    for record_name in  test_name:
        input_file = test_path+record_name    
        data= pd.read_csv(input_file,sep='|')  
        data=np.array(data.iloc[:,1:-3])

        num_rows = len(data)
        scores = np.zeros(num_rows)
        labels = np.zeros(num_rows)
        for t in range(num_rows):  
            raw_data = data[:t+1]    
            current_score, current_label = get_sepsis_score(raw_data,model,k)
            scores[t] = current_score
            labels[t] = current_label 

        record_name=record_name[:-4]
        output_file = output_path+record_name + '.psv'
        save_challenge_predictions(output_file, scores, labels)    

    auroc, auprc, accuracy, f_measure, utility = evaluate_scores(test_path, output_path)
    print('cutoff is '+str(k))
    output_string = 'AUROC|AUPRC|Accuracy|F-measure|Utility\n{}|{}|{}|{}|{}'.format(auroc, auprc, accuracy, f_measure, utility)
    print(output_string)

processing the 0 person
processing the 1000 person
processing the 2000 person
processing the 3000 person
processing the 4000 person
processing the 5000 person
processing the 6000 person
processing the 7000 person
processing the 8000 person
cutoff is 8066
AUROC|AUPRC|Accuracy|F-measure|Utility
0.7963271023904925|0.09172217207448993|0.760414365790493|0.09346156641391506|0.316248188151681
processing the 0 person
processing the 1000 person
processing the 2000 person
processing the 3000 person
processing the 4000 person
processing the 5000 person
processing the 6000 person
processing the 7000 person
processing the 8000 person
cutoff is 8066
AUROC|AUPRC|Accuracy|F-measure|Utility
0.7963271023904925|0.09172217207448993|0.7768462230417343|0.0971623214308842|0.3258901435451443
processing the 0 person
processing the 1000 person
processing the 2000 person
processing the 3000 person
processing the 4000 person
processing the 5000 person
processing the 6000 person
processing the 7000 person
processi